# Task 2: Preprocessing for Text-to-Image with CLIP Tokenizer

This notebook demonstrates how to use the CLIP tokenizer to preprocess text for text-to-image models like Stable Diffusion.

## What is CLIP?
CLIP (Contrastive Language-Image Pre-training) is a neural network trained on a variety of image-text pairs. It can be used to understand both images and text in a shared embedding space, allowing for tasks like zero-shot image classification and text-to-image generation.

## Why Use CLIP for Text-to-Image?
Text-to-image models like Stable Diffusion use CLIP's text encoder to understand and represent text prompts in a way that guides the image generation process. The CLIP tokenizer is the first step in this pipeline, converting raw text into tokens that the CLIP text encoder can process.

## Step 1: Install Required Libraries

In [ ]:
!pip install transformers torch diffusers ftfy

## Step 2: Import Libraries

In [ ]:
import torch
from transformers import CLIPTokenizer, CLIPTextModel
import matplotlib.pyplot as plt
import numpy as np

## Step 3: Load CLIP Tokenizer

We'll load the CLIP tokenizer from the specified model: 'openai/clip-vit-base-patch32'

In [ ]:
# Load the CLIP tokenizer
model_id = "openai/clip-vit-base-patch32"
tokenizer = CLIPTokenizer.from_pretrained(model_id)
print(f"Tokenizer loaded: {tokenizer.__class__.__name__}")

## Step 4: Tokenize Sample Text

Let's tokenize a sample text prompt that might be used for text-to-image generation.

In [ ]:
# Sample prompt
prompt = "A scenic mountain landscape during sunset"
print(f"Input prompt: '{prompt}'")

# Tokenize the text
tokens = tokenizer.tokenize(prompt)
print(f"\nTokenized text: {tokens}")
print(f"Number of tokens: {len(tokens)}")

# Show the mapping from tokens to their IDs
print("\nToken to ID mapping:")
for token in tokens:
    token_id = tokenizer.convert_tokens_to_ids(token)
    print(f"  '{token}' -> {token_id}")

## Step 5: Encode the Text for CLIP Model Input

Now, let's encode the text prompt to create inputs for the CLIP text encoder.

In [ ]:
# Encode the text
text_inputs = tokenizer(prompt, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")

print("Encoded input for CLIP:")
for key, value in text_inputs.items():
    print(f"\n{key}: {value}")
    
# Show the full tokenization with special tokens
input_ids = text_inputs["input_ids"][0]
tokens_with_special = tokenizer.convert_ids_to_tokens(input_ids)
print(f"\nFull tokenization with special tokens: {tokens_with_special}")

# Count non-padding tokens
non_padding_tokens = [token for token in tokens_with_special if token != tokenizer.pad_token]
print(f"Number of non-padding tokens: {len(non_padding_tokens)}")

## Step 6: Understanding the CLIP Tokenizer

The CLIP tokenizer has some similarities and differences compared to other tokenizers like BERT's.

In [ ]:
# Let's look at some details of the CLIP tokenizer
print(f"Vocabulary size: {tokenizer.vocab_size}")
print(f"Model max length: {tokenizer.model_max_length}")
print(f"Beginning of string (BOS) token: {tokenizer.bos_token} (ID: {tokenizer.bos_token_id})")
print(f"End of string (EOS) token: {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")
print(f"Padding token: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")

## Step 7: Generate Text Embeddings Using CLIP Text Encoder

Now, let's use the CLIP text encoder to generate embeddings from our tokenized prompt.

In [ ]:
# Load the CLIP text encoder
text_encoder = CLIPTextModel.from_pretrained(model_id)

# Set the model to evaluation mode
text_encoder.eval()

# Generate text embeddings
with torch.no_grad():
    text_outputs = text_encoder(**text_inputs)
    
# Get the text embeddings
text_embeddings = text_outputs.last_hidden_state
print(f"Text embeddings shape: {text_embeddings.shape}")
print("This represents: [batch_size, sequence_length, hidden_size]")

# The last token's embedding is often used as the sentence embedding in CLIP
sentence_embedding = text_embeddings[0, -1, :]
print(f"\nSentence embedding shape: {sentence_embedding.shape}")
print(f"Sample of sentence embedding (first 10 values): {sentence_embedding[:10]}")

## Step 8: Preparing Text Embeddings for Stable Diffusion

For text-to-image models like Stable Diffusion, we need to prepare the text embeddings in a specific format.

In [ ]:
# Let's also process a negative prompt (what we don't want in the image)
negative_prompt = "blurry, bad art, sketchy, unclear"
negative_inputs = tokenizer(negative_prompt, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")

with torch.no_grad():
    negative_outputs = text_encoder(**negative_inputs)
    negative_embeddings = negative_outputs.last_hidden_state

# For Stable Diffusion, we need to concatenate positive and negative embeddings
# The model will try to generate images that match the positive embeddings and avoid the negative ones
combined_embeddings = torch.cat([negative_embeddings, text_embeddings])
print(f"Combined embeddings shape: {combined_embeddings.shape}")

## Step 9: Visualizing Text Embeddings

Let's visualize some aspects of the text embeddings to better understand them.

In [ ]:
# Convert the embeddings to numpy arrays for easier manipulation
positive_embeddings_np = text_embeddings[0].detach().numpy()
negative_embeddings_np = negative_embeddings[0].detach().numpy()

# Visualize the embeddings as a heatmap for selected tokens
plt.figure(figsize=(15, 5))

# Show only a subset of the dimensions for clarity
dims_to_show = 50
tokens_to_show = min(10, len(tokens_with_special))

plt.imshow(positive_embeddings_np[:tokens_to_show, :dims_to_show], aspect='auto', cmap='viridis')
plt.colorbar(label='Embedding Value')
plt.title('CLIP Text Embeddings Visualization (First 50 Dimensions)')
plt.xlabel('Embedding Dimension')
plt.ylabel('Token Position')

# Add token labels
plt.yticks(range(tokens_to_show), tokens_with_special[:tokens_to_show])
plt.xticks(range(0, dims_to_show, 10))
plt.grid(False)
plt.tight_layout()
plt.show()

## Step 10: Comparing Different Prompts

Let's compare embeddings for different prompts to see how they differ.

In [ ]:
# Let's process a different prompt
prompt2 = "A futuristic cityscape with flying cars at night"
inputs2 = tokenizer(prompt2, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")

with torch.no_grad():
    outputs2 = text_encoder(**inputs2)
    embeddings2 = outputs2.last_hidden_state

# Get the sentence embeddings (last token)
sentence_embedding1 = text_embeddings[0, -1, :].detach().numpy()
sentence_embedding2 = embeddings2[0, -1, :].detach().numpy()

# Calculate cosine similarity between the two embeddings
cos_sim = np.dot(sentence_embedding1, sentence_embedding2) / (np.linalg.norm(sentence_embedding1) * np.linalg.norm(sentence_embedding2))

print(f"Prompt 1: '{prompt}'")
print(f"Prompt 2: '{prompt2}'")
print(f"Cosine similarity between sentence embeddings: {cos_sim:.4f}")
print(f"The similarity score ranges from -1 to 1, where 1 means identical and -1 means opposite.")

## Conclusion

In this notebook, we've demonstrated how to:

1. Load and use the CLIP tokenizer from Hugging Face
2. Tokenize and encode text prompts for text-to-image generation
3. Generate text embeddings using the CLIP text encoder
4. Prepare embeddings in a format suitable for text-to-image models like Stable Diffusion
5. Visualize and compare text embeddings

These preprocessing steps are crucial for text-to-image generation with models like Stable Diffusion, as they convert natural language prompts into the numerical representations needed by the diffusion models to guide the image generation process.

Next steps would typically involve feeding these embeddings into a text-to-image diffusion model to generate images based on the text prompts.